In [ ]:
#Importing necessary packages and libraries
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import train_test_split
from matplotlib import cm
import numpy as np
import pickle
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.naive_bayes import MultinomialNB

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [4]:
from nltk.tokenize import RegexpTokenizer
import re
import string
from nltk.stem import WordNetLemmatizer 

In [5]:
#scraped CSV into dataframe
df=pd.read_csv('file.csv')
df.shape

/Users/jeffreyng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(135742, 35)

In [6]:
df[df.language=='en'].to_csv('english_results.csv')

In [7]:
df1=pd.read_csv('LIWC_English_results.csv')

/Users/jeffreyng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (6,7,16,17,18,24,25,26,27,28,29,30,32,33,34,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
data=df[df.language=='en']

In [9]:
data.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'language', 'mentions',
       'urls', 'photos', 'replies_count', 'retweets_count', 'likes_count',
       'hashtags', 'cashtags', 'link', 'retweet', 'quote_url', 'video', 'near',
       'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to',
       'retweet_date', 'translate', 'trans_src', 'trans_dest'],
      dtype='object')

In [ ]:
df3=pd.concat([df1, data], axis=1 )

In [ ]:
df3=df3[['username','Tone', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'tweet']]

In [ ]:
df3

In [ ]:
df3[df3.sad>20].tweet

In [ ]:
df3=df3.dropna(axis=0)

In [ ]:
df3[df3.sad>15]

In [ ]:
data3=pd.read_csv('suicide.csv')
data1=data[data.language=='en']

In [ ]:
data1.tweet

In [ ]:
data2=pd.read_csv('lonely_results.csv')
data2

In [ ]:
lonely_data=pd.concat([data2,data1], axis=1)

In [ ]:
lonely_data=lonely_data[lonely_data.language=='en']

In [ ]:
lonely_data= lonely_data[['username','Tone', 'posemo', 'negemo', 'anx', 'anger', 'sad', 'tweet']]

In [ ]:
final_data=pd.concat([lonely_data, df3], axis=0)

In [ ]:
final_data

In [ ]:
import string
sw_list = stopwords.words('english')
sw_list += list(string.punctuation)
sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘', '©', '#', '@', 
            'said', 'one', 'com', '-', '–', '—', ' ', 'bizpacreview', 'ccya4ewhiyi' ,'8bkua22xbjnl']
sw_list += ['www', 'the', 'to', 'and', 'a', 's', 't','is', 'you', 'of', 'i', 'u'
            'in', 'this', 'are', 'it', 'for', 'https', 'com', 'twitter']
sw_list += [str(x) for x in range(100)]
sw_list += [int(x) for x in range(100)]
sw_set = set(sw_list)

In [ ]:
#using tf_idf vectorizer
tfidf = TfidfVectorizer(stop_words=sw_set, ngram_range=(1,1), lowercase=True)
# tfidf_data_train_lem = tfidf.fit_transform()
answer = tfidf.fit_transform(final_data['tweet'])

In [ ]:
answer

In [ ]:
df_wordvec = pd.DataFrame(answer.todense(), columns = tfidf.get_feature_names())
df_wordvec

In [ ]:
# Mapped Vader scores to integer counts for easier manipulation 
col         = 'sad'
conditions  = [ final_data[col] >= 5, (final_data[col] < 5)]
choices     = [ 'depressed', 'not_depressed' ]
final_data["target"] = np.select(conditions, choices, default=np.nan)            


In [ ]:
sns.barplot(x=['depressed', 'not depressed'], y=[final_data['target'].value_counts()[1], final_data['target'].value_counts()[0]])

In [ ]:
def NLP_process(data):
    BoW=[]
    for word in data:
        BoW.append(word)
    word_bank= ' '.join(BoW)
    word_bank= word_bank.lower()
#     from nltk.tokenize import RegexpTokenizer
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    tokenized_tweets= tokenizer.tokenize(word_bank)

    sw_list = stopwords.words('english')
    sw_list += list(string.punctuation)
    sw_list += ["''", '""', '...', '``', '’', '“', '’', '”', '‘', '‘', '©', '#', '@', 
            'said', 'one', 'com', '-', '–', '—', '.']
    sw_list += ['www', 'the', 'to', 'and', 'a', 's', 't','is', 'you', 'of', 'i', 
            'in', 'this', 'are', 'it', 'for', 'https', 'com', 'twitter', 'alllivesmatter' , 
            'blacklivesmatter' , 'bluelivesmatter', 'p', 'status', 'feel']
    sw_set = set(sw_list)
    filtered_tweets=[]
    for word in tokenized_tweets:
        if word not in sw_set:
            filtered_tweets.append(word)
#     
    lemmatizer=WordNetLemmatizer()
    X_lemmatized_tweets=[]
    for w in filtered_tweets:
        X_lemmatized_tweets.append(lemmatizer.lemmatize(w))
    return (X_lemmatized_tweets)

In [ ]:
z=NLP_process(final_data['tweet'])


In [ ]:
fdist=FreqDist(z)
plt.figure(figsize=(10,10))
fdist.plot(35)

In [ ]:
depressed=final_data[final_data['target']==1]
depressed['tweet']

In [ ]:
not_depressed=final_data[final_data['target']==0]
not_depressed['tweet']

In [ ]:
a=NLP_process(depressed['tweet'])

In [ ]:
b=NLP_process(not_depressed['tweet'])

In [ ]:
depressed = [item for sublist in a for item in sublist]
not_depressed = [item for sublist in b for item in sublist]

In [ ]:
depressed_list=FreqDist(a)
not_depressed_list=FreqDist(b)

In [ ]:
depressed_list
not_depressed_list

In [ ]:
not_depressed_bar_counts = [x[1] for x in not_depressed_list.most_common(50)]
not_depressed_bar_words = [x[0] for x in not_depressed_list.most_common(50)]

In [ ]:
depressed_bar_counts = [x[1] for x in depressed_list.most_common(50)]
depressed_bar_words = [x[0] for x in depressed_list.most_common(50)]

In [ ]:
from wordcloud import WordCloud
D_dictionary = dict(zip(depressed_bar_words, depressed_bar_counts))
ND_dictionary = dict(zip(not_depressed_bar_words, not_depressed_bar_counts))

In [ ]:
# Create the word cloud:

wordcloud = WordCloud(colormap='Spectral').generate_from_frequencies(D_dictionary)

# Display the generated image w/ matplotlib:

plt.figure(figsize=(10,10), facecolor='k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
# plt.savefig('pos_Wcloud.png')
# Uncomment the next line if you want to save your image:
# plt.savefig('satire_wordcloud.png')

plt.show()

In [ ]:
wordcloud = WordCloud(colormap='Spectral').generate_from_frequencies(ND_dictionary)

# Display the generated image w/ matplotlib:

plt.figure(figsize=(10,10), facecolor='k')
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.tight_layout(pad=0)
# plt.savefig('pos_Wcloud.png')
# Uncomment the next line if you want to save your image:
# plt.savefig('satire_wordcloud.png')

plt.show()

In [ ]:
X=final_data['tweet']
y=final_data['target']

In [ ]:
#train, test, split
X_train, y_train, X_test, y_test= train_test_split(X, y, test_size=.20, random_state=0)

In [ ]:
#using tf_idf vectorizer
tfidf = TfidfVectorizer(stop_words=sw_set, ngram_range=(1,1), lowercase=True)
# tfidf_data_train_lem = tfidf.fit_transform()
X_b = tfidf.fit_transform(X)
# X_test_b=tfidf.transform(X_test)

In [ ]:
df=pd.DataFrame(X_b.todense(), columns=tfidf.get_feature_names())

In [ ]:
df

In [ ]:
rfc=RandomForestClassifier(n_jobs=-1, random_state=0)

In [ ]:
rfc.fit(df, y_train)

In [ ]:
rfc.predict()

In [ ]:
len(y_train)